In [2]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


## Holography

In [5]:
import numpy as np
import cupy as cp
import gpie
from gpie import Graph, SupportPrior, ConstWave, fft2, AmplitudeMeasurement, mse
from gpie.core.linalg_utils import circular_aperture, masked_random_array
from numpy.typing import NDArray

gpie.set_backend(np)
# ==== Setting Parameter ====
H, W = 256, 256
shape = (H, W)
noise = 1e-4
rng = np.random.default_rng(seed=42)
support_x = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
data_x = masked_random_array(support_x, dtype=np.complex64, rng=rng)
support_y =  circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))


class Holography_with_Uncertainty(Graph):
    def __init__(self, var: np.float64,  ref_wave: NDArray[np.complex64], support: NDArray[np.bool_]) -> None:
        super().__init__()
        obj = ~SupportPrior(support=support, label="obj", dtype = np.complex64)
        with self.observe():
            meas = AmplitudeMeasurement(var=var) @ (fft2(ref_wave + obj))
        self.compile()


g = Holography_with_Uncertainty(var = noise, ref_wave = data_x, support = support_y)
g.set_init_rng(np.random.default_rng(seed=11))
g.generate_sample(rng=np.random.default_rng(seed=9), update_observed=True)

true_obj = g.get_wave("obj").get_sample()


def monitor(graph, t):
    x = graph.get_wave("obj").compute_belief().data
    err = mse(x, true_obj)
    if t % 10 == 0:
        print(f"[t={t}] PSE (sum) = {err:.5e}")

g.run(n_iter=100, callback=monitor, verbose = False)


[t=0] PSE (sum) = 4.28104e-01
[t=10] PSE (sum) = 1.31072e-03
[t=20] PSE (sum) = 8.49448e-04
[t=30] PSE (sum) = 8.01702e-04
[t=40] PSE (sum) = 8.01840e-04
[t=50] PSE (sum) = 8.01880e-04
[t=60] PSE (sum) = 8.01892e-04
[t=70] PSE (sum) = 8.01895e-04
[t=80] PSE (sum) = 8.01896e-04
[t=90] PSE (sum) = 8.01896e-04


In [6]:
gpie.set_backend(cp)
g.to_backend()

In [8]:
g.run(n_iter=100, callback=None, verbose = False)

## Holography with reference-wave correction

In [42]:
import numpy as np
from gpie import Graph, SupportPrior, ConstWave, fft2, AmplitudeMeasurement, pmse, mse
from gpie.core.linalg_utils import circular_aperture, masked_random_array
from numpy.typing import NDArray

# ==== Setting Parameter ====
H, W = 128, 128
shape = (H, W)
noise = 1e-4
rng = np.random.default_rng(seed=42)
support_ref = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
ref = masked_random_array(support_x, dtype=np.complex64, rng=rng)
support_obj =  circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))


class Holography(Graph):
    def __init__(self, var: np.float64,  ref: NDArray[np.complex64], support: NDArray[np.bool_]) -> None:
        super().__init__()
        obj = ~SupportPrior(support=support, label="obj", dtype = np.complex64)
        ref_wave = ~ConstWave(data = ref, large_value=1e3, label = "ref")
        with self.observe():
            meas = AmplitudeMeasurement(var=var, damping = 0.0) @ (fft2(ref_wave + obj))
        self.compile()


g = Holography(var = noise, ref = ref, support = support_obj)
g.set_init_rng(np.random.default_rng(seed=11))
g.generate_sample(rng=np.random.default_rng(seed=9), update_observed=True)

true_obj = g.get_wave("obj").get_sample()


def monitor(graph, t):
    x = graph.get_wave("obj").compute_belief().data
    err = mse(x, true_obj)
    if t % 10 == 0:
        print(f"[t={t}] PMSE (sum) = {err:.5e}")

g.run(n_iter=100, callback=monitor, verbose = False)


[t=0] PMSE (sum) = 7.01359e-01
[t=10] PMSE (sum) = 7.83272e-04
[t=20] PMSE (sum) = 6.96995e-04
[t=30] PMSE (sum) = 6.97818e-04
[t=40] PMSE (sum) = 6.97858e-04
[t=50] PMSE (sum) = 6.97861e-04
[t=60] PMSE (sum) = 6.97861e-04
[t=70] PMSE (sum) = 6.97861e-04
[t=80] PMSE (sum) = 6.97861e-04
[t=90] PMSE (sum) = 6.97861e-04


In [34]:
print(-10 * np.log10(np.linalg.norm(ref - g.get_wave("ref").get_sample())/np.prod(ref.shape)))

36.04757222954459


In [35]:
-10 * np.log10(np.linalg.norm(ref - g.get_wave("ref").compute_belief().data)/np.prod(ref.shape))

np.float64(50.92511796920036)